In [ ]:
import requests

def search_scopus_author(query, api_key):
    url = "https://api.elsevier.com/content/search/author"
    headers = {
        "X-ELS-APIKey": api_key
    }
    params = {
        "query": query
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Check for any HTTP errors
        data = response.json()

        # Parse the response and extract author information
        authors = data.get("search-results", {}).get("entry", [])

        if authors:
            print(f"Found {len(authors)} authors matching the query '{query}':")
            for author in authors:
                author_name = author.get("preferred-name", {}).get("indexed-name", "")
                author_id = author.get("dc:identifier", "").split(":")[-1]
                print(f"- {author_name} (Scopus Author ID: {author_id})")

                # Retrieve co-authors for the current author
                coauthor_url = f"https://api.elsevier.com/content/author/co-author?author_id={author_id}"
                coauthor_response = requests.get(coauthor_url, headers=headers)
                coauthor_data = coauthor_response.json()

                coauthors = coauthor_data.get("author-retrieval-response", {}).get("co-authors", [])
                if coauthors:
                    print("  Co-Authors:")
                    for coauthor in coauthors:
                        coauthor_name = coauthor.get("preferred-name", {}).get("indexed-name", "")
                        coauthor_id = coauthor.get("author-url", "").split("/")[-1]
                        print(f"    - {coauthor_name} (Scopus Author ID: {coauthor_id})")
                else:
                    print("  No co-authors found.")

        else:
            print(f"No authors found matching the query '{query}'.")

    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Replace these with your actual query and API key
search_query = "John Smith"  # Example author name to search for
api_key = "af1361e4e58683804f723b56fed20175"  # Your Scopus API key

search_scopus_author(search_query, api_key)


: 